In [3]:
"""This may be useless since ownership data is pretty delayed. Might be a good candidate for processing stream 
data (stock orders) to look for big insitutaional trades (Basically huge spikes in volume?)
Better idea is tracking insiders via FinViz Scraping.
"""
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import os
import pandas as pd
import datetime


In [11]:
"""Get data from FinViz and parse to list of lists"""
#this url is just for the buys, if you replace tc=1 with tc=2 it will be the sells and tc=7 for all
finviz_url = 'https://finviz.com/insidertrading.ashx?tc=1'

req = Request(url=finviz_url,headers={'user-agent': 'my-app/0.0.1'}) 
response = urlopen(req)    

# Read the contents of the file into 'html'
html = BeautifulSoup(response)


# Find gold in the Soup
buys1 = html.body.find_all('tr', attrs={'class':'insider-buy-row-1'})
buys2 = html.body.find_all('tr', attrs={'class':'insider-buy-row-2'})
buys1.extend(buys2)

buy_data = []
for buy in buys1:
    inner_list = []
    if str(buy.find_all('td', attrs={'style':'white-space:nowrap'})[3]).split('>')[1].split('</td')[0].lower() == 'buy':
    
        ticker = str(buy.find('a')).split('>')[1].split('</a')[0]
        inner_list += [ticker]

        date = str(buy.find_all('td', attrs={'style':'white-space:nowrap'})[2]).split('>')[1].split('</td')[0]
        inner_list += [date]

        filing = str(buy.find_all('td', attrs={'style':'white-space:nowrap'})[4]).split('>')[2].split('</a')[0]
        inner_list += [filing]
        
        cost = str(buy.find_all('td', attrs={'align':'right'})[0]).split('>')[1].split('</td')[0]
        inner_list += [cost]
        
        shares = str(buy.find_all('td', attrs={'align':'right'})[1]).split('>')[1].split('</td')[0]
        inner_list += [shares]
        
        value = str(buy.find_all('td', attrs={'align':'right'})[2]).split('>')[1].split('</td')[0]
        inner_list += [value]
        
        shares_total = str(buy.find_all('td', attrs={'align':'right'})[3]).split('>')[1].split('</td')[0]
        inner_list += [shares_total]
        
        buy_data += [inner_list] 
    else:
        pass
    

In [13]:
"""Create DataFrame and fix up some columns (the date column is a little funky since it comes as %b %d)"""
cols = ['ticker','date', 'filing', 'cost', 'shares', 'value', 'shares_total']
df_buys = pd.DataFrame(buy_data, columns = cols) 

if datetime.datetime.utcnow() - datetime.datetime(2020, 12, 1) < datetime.datetime(2021, 12, 1) - datetime.datetime.utcnow():
    df_buys.loc[:,"date"] = [datetime.datetime.strptime(x+' 2020','%b %d %Y') for x in df_buys["date"]]
    df_buys.loc[:,"filing"] = [datetime.datetime.strptime(x+' 2020','%b %d %I:%M %p %Y') for x in df_buys["filing"]]
else:
    df_buys.loc[:,"date"] = [datetime.datetime.strptime(x+' 2021','%b %d %Y') for x in df_buys["date"]]
    df_buys.loc[:,"filing"] = [datetime.datetime.strptime(x+' 2021','%b %d %I:%M %p %Y') for x in df_buys["filing"]]

df_buys.loc[:,"cost"] = df_buys["cost"].astype(float)
df_buys.loc[:,"shares"] = [pd.to_numeric(x.replace(',', '')) for x in df_buys["shares"]]
df_buys.loc[:,"value"] = [pd.to_numeric(x.replace(',', '')) for x in df_buys["value"]]
df_buys.loc[:,"shares_total"] = [pd.to_numeric(x.replace(',', '')) for x in df_buys["shares_total"]]
df_buys.head()

,ticker,date,filing,cost,shares,value,shares_total
0,MMLP,2020-10-09,2020-10-13 12:29:00,1.29,1426,1846,108173
1,MMLP,2020-10-09,2020-10-13 12:29:00,1.29,884,1145,19786
2,MMLP,2020-10-09,2020-10-13 12:29:00,1.29,2184,2827,454285
3,EYEN,2020-10-12,2020-10-13 11:54:00,3.55,100000,355000,4219748
4,RCG,2020-10-09,2020-10-13 11:14:00,1.35,675,911,45984


In [21]:
"""An extra couple of cells for adding a table to the database for this information. 
Would be worthwhile to alter the design to discourage duplicates before moving on."""
import psycopg2 as pg

conn = pg.connect("dbname=StonksGoUp user=postgres host=localhost password=admin")
cur = conn.cursor()

SQL_insider = """
     DROP TABLE insidertrades;
     CREATE TABLE insidertrades (
         tradeid SERIAL,
         ticker varchar(5) NOT NULL,
         date timestamp with time zone NOT NULL,
         filing timestamp with time zone NOT NULL,
         cost numeric NOT NULL,
         shares integer NOT NULL,
         value numeric NOT NULL,
         sharestotal integer NOT NULL,
         createddate timestamp with time zone NOT NULL DEFAULT NOW(),
         CONSTRAINT pk_insidertrades PRIMARY KEY (tradeid)
     );
     """
cur.execute(SQL_insider, conn)
conn.commit()

# cur.close()
# conn.close()

In [22]:
insert = [list(row) for row in df_buys.itertuples(index=False)]

SQL_insider_insert = """ INSERT INTO public.insidertrades(ticker, date, filing, cost, shares, value, sharestotal) 
    VALUES (%s, %s, %s, %s, %s, %s, %s) ON CONFLICT DO NOTHING"""
cur.executemany(SQL_insider_insert, insert)
conn.commit()

print(f'{cur.rowcount} rows inserted.')
cur.close()
conn.close()

132 rows inserted.
